In [ ]:
!pip install datasets
!pip install evaluate
!pip install accelerate -U
!pip install peft
!pip install --upgrade pyarrow
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.8 M

In [ ]:
import torch
import evaluate
import numpy as np
import pandas as pd
from transformers import (AutoModelForSequenceClassification, AutoTokenizer, BitsAndBytesConfig)
from datasets import (load_dataset, Dataset)
from torch.utils.data import DataLoader
from torch.optim import AdamW
from peft import LoraConfig, get_peft_model
from google.colab import drive

In [ ]:
torch.manual_seed(42)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
import gc
import os

def clear_cache():
  for _ in range(5):
    gc.collect()
    torch.cuda.empty_cache()
    os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
data_dir = "/content/drive/MyDrive/"
# data_csv = data_dir + "Subset2.csv"

In [ ]:
train_df = pd.read_csv(data_dir+ "train.csv")
validation_df = pd.read_csv(data_dir + "test.csv")

train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)


In [ ]:
# prompt: do a loop that goes over both datasets that we created and prints the number of post with column label = 1 and column label = 0... print also the name (Train, validation and test) of the dataset

for dataset, name in zip([train_dataset, validation_dataset, ], ["Train", "Validation", ]):
  num_label_1 = len(dataset.filter(lambda example: example['label'] == 1))
  num_label_0 = len(dataset.filter(lambda example: example['label'] == 0))
  print(f"{name} Dataset:")
  print(f"Number of posts with label 1: {num_label_1}")
  print(f"Number of posts with label 0: {num_label_0}")
  print("----")


Filter:   0%|          | 0/36640 [00:00<?, ? examples/s]

Filter:   0%|          | 0/36640 [00:00<?, ? examples/s]

Train Dataset:
Number of posts with label 1: 18012
Number of posts with label 0: 18628
----


Filter:   0%|          | 0/12214 [00:00<?, ? examples/s]

Filter:   0%|          | 0/12214 [00:00<?, ? examples/s]

Validation Dataset:
Number of posts with label 1: 5965
Number of posts with label 0: 6249
----


In [ ]:
LLAMA = "meta-llama/Meta-Llama-3-8B"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    return tokenizer(
        example['body'],
        padding="max_length",
        truncation=True,
        max_length=512  # Adjust max_length based on your needs
    )

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = validation_dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/36640 [00:00<?, ? examples/s]

Map:   0%|          | 0/12214 [00:00<?, ? examples/s]

In [ ]:
def format_labels(example):
    example['labels'] = example['label']
    return example

tokenized_train_dataset = tokenized_train_dataset.map(format_labels)
tokenized_train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

tokenized_val_dataset = tokenized_val_dataset.map(format_labels)
tokenized_val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


Map:   0%|          | 0/36640 [00:00<?, ? examples/s]

Map:   0%|          | 0/12214 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model = AutoModelForSequenceClassification.from_pretrained(
    "meta-llama/Meta-Llama-3-8B",
    num_labels=2,
    quantization_config=quantization_config,
    device_map="auto",
    pad_token_id=tokenizer.pad_token_id,
    offload_folder="content/drive/MyDrive/offload", # Specify a folder to offload parts of the model to disk
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    modules_to_save=["score"],
    task_type=TaskType.SEQ_CLS,
)

model = get_peft_model(model, lora_config)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions, eval_pred.label_ids
    preds = np.argmax(predictions, axis=1)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    accuracy = accuracy_score(labels, preds)
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall}

In [ ]:
BATCH_SIZE = 32

In [ ]:
clear_cache()

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    fp16=True,
    save_total_limit=3,
    save_strategy="epoch",
    label_names=["labels"],
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two 

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall
1,0.307100,0.350293,0.835271,0.856447,0.796144


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommend

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall
1,0.307100,0.350293,0.835271,0.856447,0.796144
2,0.296900,0.342187,0.846488,0.931253,0.740319
3,0.279200,0.329353,0.849844,0.877812,0.804526


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


TrainOutput(global_step=3435, training_loss=0.37458488885665736, metrics={'train_runtime': 21538.5567, 'train_samples_per_second': 5.103, 'train_steps_per_second': 0.159, 'total_flos': 2.3614356406350643e+18, 'train_loss': 0.37458488885665736, 'epoch': 3.0})

In [ ]:
# Save the fine-tuned model
trainer.save_model("/content/drive/MyDrive/fine_tuned_llama")